# Experiment 8

# Aim: Implementation of TransferLearning(VGG16,VGG19)

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam



In [2]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the data to be in the range [0,1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to categorical format
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


In [3]:
# Input shape
input_shape = x_train.shape[1:]


# Define VGG19 Model for Transfer Learning


In [4]:
def build_vgg19_model():
    # Load the VGG19 model without the top classification layer (include_top=False)
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze the base model's layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom classification layers on top of the VGG19 base
    x = Flatten()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dense(10, activation='softmax')(x)

    # Create the final model
    model = Model(inputs=base_model.input, outputs=x)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Build the model
vgg19_model = build_vgg19_model()
vgg19_model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 32, 32, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 32, 32, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 16, 16, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 16, 16, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 8, 8, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 8, 8, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 8, 8, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv4 (Conv2D)                │ (None, 8, 8, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 4, 4, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv4 (Conv2D)                │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 2, 2, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 2, 2, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 2, 2, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 2, 2, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv4 (Conv2D)                │ (None, 2, 2, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 20,158,282 (76.90 MB)

 Trainable params: 133,898 (523.04 KB)

 Non-trainable params: 20,024,384 (76.39 MB)

# Train VGG19 Model

In [5]:
# Training VGG19
print("Training VGG19 Model...")
vgg19_history = vgg19_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

Training VGG19 Model...
Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 435s 550ms/step - accuracy: 0.4431 - loss: 1.5706 - val_accuracy: 0.5463 - val_loss: 1.3014
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 455s 581ms/step - accuracy: 0.5646 - loss: 1.2411 - val_accuracy: 0.5710 - val_loss: 1.2285
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 445s 569ms/step - accuracy: 0.5849 - loss: 1.1786 - val_accuracy: 0.5711 - val_loss: 1.2216
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 488s 624ms/step - accuracy: 0.6002 - loss: 1.1330 - val_accuracy: 0.5883 - val_loss: 1.1807
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 466s 596ms/step - accuracy: 0.6158 - loss: 1.0839 - val_accuracy: 0.5901 - val_loss: 1.1700
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 491s 581ms/step - accuracy: 0.6321 - loss: 1.0526 - val_accuracy: 0.5922 - val_loss: 1.1671
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 424s 543ms/step - accuracy: 0.6367 - loss: 1.0247 - val_accuracy: 0.6026 - val_loss: 1.1439
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 449s 573ms/s

# Model Evaluation

In [6]:
# Evaluate VGG19
vgg19_test_loss, vgg19_test_acc = vgg19_model.evaluate(x_test, y_test, verbose=2)
print(f"VGG19 Test Accuracy: {vgg19_test_acc * 100:.2f}%")


313/313 - 45s - 145ms/step - accuracy: 0.6086 - loss: 1.1357
VGG19 Test Accuracy: 60.86%


# Make Prediction On Test Data

In [7]:
# Predictions using VGG19
vgg19_predictions = vgg19_model.predict(x_test)
vgg19_pred_classes = tf.argmax(vgg19_predictions, axis=1)

print(f"VGG19 Predictions: {vgg19_pred_classes[:10]}")


313/313 ━━━━━━━━━━━━━━━━━━━━ 43s 136ms/step
VGG19 Predictions: [3 8 9 9 6 4 1 6 4 3]


# By IT154 Ashish Vinod